In [1]:
import os
import pandas as pd
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

## 1. Get the sample of parsed AVL data.
Clean MTA prefixes from trip_id

In [2]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('spark_parse/test2.txt',header=None)

# beware!  Bonan is still working on organizing the extract files.  these columns may change.
bustime.columns = ['route','lat','lon','timestamp','vehicle_id','trip_id','trip_date','shape_id',
                   'next_stop_id','dist_from_stop','stop_dist_on_trip','presentable_dist']

bustime.drop_duplicates(['vehicle_id','timestamp'],inplace=True)
bustime['trip_id'] = bustime['trip_id'].str.replace('MTA NYCT_','')
bustime['trip_id'] = bustime['trip_id'].str.replace('MTABC_','')
bustime.set_index(['route','trip_id','trip_date','vehicle_id'],inplace=True,drop=True)

/opt/rh/anaconda/root/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
# for demonstration, use a subset. Just get data for one line (B41) on one day.
tripDateLookup = "2015-01-06"
lineLookup = "MTA NYCT_B41"
bustime = bustime.xs((lineLookup,tripDateLookup),level=(0,2),
                           drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
bustime['timestamp'] = bustime['timestamp'].apply(ttools.parseActualTime,tdate='2015-01-06')
print 'Finished loading BusTime data, parsing timestamps, and and slicing B41 line.'

Finished loading BusTime data, parsing timestamps, and and slicing B41 line.


## 2. Convert distance fields to numeric and make a calculated column for "distance along trip"
Also show range of calculated value.  B41 route run the entire length of Flatbush Avenue, approximately 8.5 miles.

In [4]:
bustime['stop_dist_on_trip'] = bustime['stop_dist_on_trip'].convert_objects(convert_numeric=True)
bustime['dist_from_stop'] = bustime['dist_from_stop'].convert_objects(convert_numeric=True)
bustime['veh_dist_along_trip'] = bustime['stop_dist_on_trip'] - bustime['dist_from_stop']

In [7]:
print (bustime['veh_dist_along_trip'].min(),bustime['veh_dist_along_trip'].max())

(-1139.9100000000001, 12654.860000000001)


## 3. Check and investigate multiple vehicles reported on the same trip
Usually there is a clear "winner."  Perhaps this due to logging in/out of vehicles before the trip actually begins.
But sometimes there are many observations recorded from both vehicles.

In [8]:
# create a GroupBy object for convenience, since most analysis is on trip and vehicle
bg = bustime.groupby(level=(0,1,2,3))
trip_veh_validation = pd.DataFrame(bg.size(),columns=['N'])
trip_veh_validation['time_range'] = bg['timestamp'].max()-bg['timestamp'].min()
trip_veh_validation['dist_range'] = bg['veh_dist_along_trip'].max()-bg['veh_dist_along_trip'].min()
trip_veh_validation.head(25)

N  \
route        trip_id                         trip_date  vehicle_id          
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1 2015-01-06 MTA NYCT_4233  51   
                                                        MTA NYCT_4615   4   
                                                        MTA NYCT_5049  10   
             FB_A5-Weekday-SDon-006200_B41_2 2015-01-06 MTA NYCT_4615  54   
                                                        MTA NYCT_5049   3   
             FB_A5-Weekday-SDon-009000_B41_1 2015-01-06 MTA NYCT_4233  39   
             FB_A5-Weekday-SDon-009700_B41_3 2015-01-06 MTA NYCT_4239  50   
             FB_A5-Weekday-SDon-012000_B41_2 2015-01-06 MTA NYCT_4615  65   
                                                        MTA NYCT_4955  70   
             FB_A5-Weekday-SDon-012200_B41_4 2015-01-06 MTA NYCT_4244  39   
             FB_A5-Weekday-SDon-015000_B41_3 2015-01-06 MTA NYCT_4239  44   
             FB_A5-Weekday-SDon-015500_B41_1 2015-01-06 MTA NYCT_4233  85   
             FB_A5-Weekday-SDon-018000_B41_4 2015-01-06 MTA NYCT_4244  40   
                                                        MTA NYCT_4955   4   
             FB_A5-Weekday-SDon-018200_B41_2 2015-01-06 MTA NYCT_4615  72   
             FB_A5-Weekday-SDon-021000_B41_1 2015-01-06 MTA NYCT_4233  41   
             FB_A5-Weekday-SDon-021500_B41_3 2015-01-06 MTA NYCT_4239  47   
             FB_A5-Weekday-SDon-024000_B41_2 2015-01-06 MTA NYCT_4229   8   
                                                        MTA NYCT_4233   2   
                                                        MTA NYCT_4615  54   
             FB_A5-Weekday-SDon-024200_B41_4 2015-01-06 MTA NYCT_4244  55   
             FB_A5-Weekday-SDon-025700_B44_3 2015-01-06 MTA NYCT_4225  27   
             FB_A5-Weekday-SDon-026000_B41_5 2015-01-06 MTA NYCT_4225   3   
                                                        MTA NYCT_4229  57   
                                                        MTA NYCT_4233   1   

                                                                       time_range  \
route        trip_id                         trip_date  vehicle_id                  
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1 2015-01-06 MTA NYCT_4233    00:52:37   
                                                        MTA NYCT_4615    00:03:09   
                                                        MTA NYCT_5049    00:09:00   
             FB_A5-Weekday-SDon-006200_B41_2 2015-01-06 MTA NYCT_4615    00:53:40   
                                                        MTA NYCT_5049    00:02:07   
             FB_A5-Weekday-SDon-009000_B41_1 2015-01-06 MTA NYCT_4233    00:35:00   
             FB_A5-Weekday-SDon-009700_B41_3 2015-01-06 MTA NYCT_4239    00:48:12   
             FB_A5-Weekday-SDon-012000_B41_2 2015-01-06 MTA NYCT_4615    00:51:04   
                                                        MTA NYCT_4955    00:56:22   
             FB_A5-Weekday-SDon-012200_B41_4 2015-01-06 MTA NYCT_4244    00:30:54   
             FB_A5-Weekday-SDon-015000_B41_3 2015-01-06 MTA NYCT_4239    00:35:55   
             FB_A5-Weekday-SDon-015500_B41_1 2015-01-06 MTA NYCT_4233    01:08:26   
             FB_A5-Weekday-SDon-018000_B41_4 2015-01-06 MTA NYCT_4244    00:30:27   
                                                        MTA NYCT_4955    00:02:38   
             FB_A5-Weekday-SDon-018200_B41_2 2015-01-06 MTA NYCT_4615    01:00:04   
             FB_A5-Weekday-SDon-021000_B41_1 2015-01-06 MTA NYCT_4233    00:34:31   
             FB_A5-Weekday-SDon-021500_B41_3 2015-01-06 MTA NYCT_4239    00:44:36   
             FB_A5-Weekday-SDon-024000_B41_2 2015-01-06 MTA NYCT_4229    00:05:46   
                                                        MTA NYCT_4233    00:00:31   
                                                        MTA NYCT_4615    00:43:30   
             FB_A5-Weekday-SDon-024200_B41_4 2015-01-06 MTA NYCT_4244    00:45:20   
             FB_A5-Weekday-SDon-02570

In [9]:
# show an example trip with multiple vehicles reporting
bustime.loc[('MTA NYCT_B41','FB_A5-Weekday-SDon-139700_B46_200','2015-01-06')].index.get_level_values(0).unique()

array(['MTA NYCT_4589', 'MTA NYCT_5010'], dtype=object)

In [10]:
stop_distances = bustime.reset_index().drop_duplicates(['shape_id','next_stop_id','stop_dist_on_trip'])
stop_distances.shape_id = stop_distances.shape_id.str.replace('MTA_','')
stop_distances.next_stop_id = stop_distances.next_stop_id.str.replace('MTA_','')
stop_distances.drop(['route','trip_id','trip_date','vehicle_id','timestamp','lat','lon','dist_from_stop',
                     'presentable_dist','veh_dist_along_trip'],axis=1,inplace=True)

In [11]:
stop_distances = stop_distances.sort(['shape_id','stop_dist_on_trip']).reset_index(drop=True)
# verify that shape_id and next_stop_id is a valid primary key (that is, exactly one value for each index)
sum(stop_distances.duplicated(['shape_id','next_stop_id']))

0

In [12]:
# show the mean distance between stops on trips
stop_distances.groupby('shape_id')['stop_dist_on_trip'].diff().mean()

375.39760814249365

## 4. Calculate speed and duration between each reported location, based on distance fields.

In [13]:
bustime['seg_speed'] = bg['veh_dist_along_trip'].diff()/(bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1))
bustime['seg_duration'] = bg['timestamp'].diff()/ttools.datetime.timedelta(seconds=1)

In [14]:
# show an example trip that contains some negative calculated speeds
bustime.loc[('MTA NYCT_B41','FB_A5-Weekday-SDon-004500_B41_1','2015-01-06','MTA NYCT_4233')]
# note that stop_dist_on_trip is not monotonically increasing

lat  \
route        trip_id                         trip_date  vehicle_id                 
MTA NYCT_B41 FB_A5-Weekday-SDon-004500_B41_1 2015-01-06 MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.613280   
                                                        MTA NYCT_4233  40.610833   
                                                        MTA NYCT_4233  40.611432   
                                                        MTA NYCT_4233  40.612922   
                                                        MTA NYCT_4233  40.614282   
                                                        MTA NYCT_4233  40.617779   
                                                        MTA NYCT_4233  40.620874   
                                                        MTA NYCT_4233  40.622216   
                                                        MTA NYCT_4233  40.624716   
                                                        MTA NYCT_4233  40.627214   
                                                        MTA NYCT_4233  40.627758   
                                                        MTA NYCT_4233  40.630890   
                                                        MTA NYCT_4233  40.632639   
                                                        MTA NYCT_4233  40.632781   
                                                        MTA NYCT_4233  40.635526   
                                                        MTA NYCT_4233  40.637535   
                                                        MTA NYCT_4233  40.640351   
                                                        MTA NYCT_4233  40.642112   
                                                        MTA NYCT_4233  40.644067   
                                                        MTA NYCT_4233  40.644723   
                                                        MTA NYCT_4233  40.646539   
                                                        MTA NYCT_4233  40.650480   
                                                        MTA NYCT_4233  40.653385   
                                                        MTA NYCT_4233  40.655210   
                                                        MTA NYCT_4233  40.658258   
                                                        MTA NYCT_4233  40.659988   
                                                        MTA NYCT_4233  40.661865   
                                                        MTA NYCT_4233  40.662940   
                                                        MTA NYCT_4233  40.666451   
                                                        MTA NYCT_4233  40.669747   
                                                        MTA NYCT_4233  40.672709   
                                                        MTA NYCT_4233  40.673025   
                                                        MTA NYCT_4233  40.676708   
                                                        MTA NYCT_4233  40.679220   
                                                        MTA NYCT_4233  40.680226   
                                                        MTA NYCT_4233  40.683093   
                                                        MTA NYCT_4233  40.686209   
                                                        MTA NYCT_4233  40.688693   
                                                        MTA NYCT_4233  40.690303   
                                                        MTA NYCT_4233  40.690859   
                                                        MTA NYCT_4233  40.691483   
                                                        MTA NYCT_4233  40.692427   
                                                        MTA NYCT_4233  40.694161   
                                                        MTA NYCT_4233  40.695354   
                                             

#### Show the scheduled sequence of stops for that trip.
Note that the first stop under 'next_stop_id', MTA_303218, is actually the third stop on the trip, then it moves backwards to the second stop.
Also note that the last stop under 'next_stop_id', MTA_307403, is not even associated with this trip.  And it comes after the last stop MTA_901007 has been reported.

In [16]:
stop_times, tz_string = gtfs.load_stop_times('gtfs_subset/')
stop_times.loc['FB_A5-Weekday-SDon-004500_B41_1']

,arrival_time,departure_time,stop_sequence,pickup_type,drop_off_type
stop_id,,,,,
303215,00:45:00,00:45:00,1,0,0
300163,00:45:52,00:45:52,2,0,0
303218,00:46:44,00:46:44,3,0,0
303219,00:47:23,00:47:23,4,0,0
306273,00:48:00,00:48:00,5,0,0
303222,00:49:00,00:49:00,6,0,0
303223,00:49:56,00:49:56,7,0,0
303224,00:50:15,00:50:15,8,0,0
303227,00:51:41,00:51:41,9,0,0


## 5. For future: identify interruptions based on speed below some threshold

In [16]:
import numpy as np
bustime['interruption_index'] = np.nan
bustime.set_index('timestamp',append=True,inplace=True)
bustime.sort_index(inplace=True)

In [17]:
def index_interrupts(df):
    counter = 0
    interruption = False
    for index, row in df.iterrows():
        if row.seg_speed > 0.4:
            interruption = False
        else:
            if interruption is False:
                counter += 1
            else:
                pass
            interruption = True
            df.loc[index,'interruption_index'] = counter
    return df

In [ ]:
bustime.groupby(level=(0,1,2,3)).apply(index_interrupts)